From the users file we can say that all the 58,842 users installed the game in the First Week of June.

Pre processed the file Purchases as our observation period is from June-August, but it has purchases from June-December
I have observed that the September-December purchases(2200) are comparitively less than that of June-August(5500).

Number of unique users who purchased any item during June-August is around 2100. So, less than 4% users actually buy inapp purchases. This has gone down to 80 (0.15%) from Sep-Dec.

Out of 58,842 around 58,831 users actually played the game atleast once from June-August (from Sessions File).

The Long term value is the is how much revenue we estimate that the user will generate over his/her lifetime playing our game.

Observation period is from June-August

The inapp purchases ranges from 3.5 to 70.

From the Purchases file I have calculated the Date of First Purchase, Date of Last Purchase and the differnce between these 2 dates, Total number of purchases made by a customer and his Long Term Revenue 

This data is really helpful because marketing people can know whether the users are interested in their game or not

All the variables like total purchases made, purchase duration and our predictor variable are Right skewed. So, we need to classify customers into High spending customers and low spending customers based on a threshold. By trial and error method I kept that threshold as 750.

From the Sessions file we got around 0.5 million observations which is a very good number as the maximum number it can reach is 
58,842(users)*91(days) = 0.535 million. So, almost all users are returning to play our game atleast one time a day during June-August.

I have created two variables namely total_sessions_played(Sum of all sessions played by an user) and Highest_Level_reached(Highest level reached by an user) during June-August.

We can classify the users as "Rookie" who are present at Right Bottom as these users played many 
sessions but did not make it to the higher levels.[total_sessions_played > 1750 and Highest_Level_reached < 50]

"Pro_Gamers" who played less sessions and made it to the higher levels [Highest_Level_reached > 25 and total_sessions_played < 250]

The rest of the users are categorized as "Normal_Gamers"


Joined these three data frames by taking unique id as udid of a customer

saved the dataframe into a csv file as a backup as our preprocessing stage is done and we just need to model our data.

We can go for Logaritmic models but it is hard to understand those interactions instead I have classified the customers on their spending habits and Gaming level. This can be useful to Marketing people as they can target their adds to incite the customers based on these classifications.

We can say that the Long Term value is in Positive linear relation with all other attributes but some are strongly correlated and some are not as we know that correlation is not a causation.

From the correlation heat map We can keep a threshold for eliminating the redundant features for building a linear regression model.

Otherwise we can go for Feature selection Backward elimination technique of Linear regression 
by keeping a threshold of confidence level to find out the most important features of this model.

Tried different models like:
Linear Regression Ordinary Least Squares model
Lasso Regression with penalization factor of 130(experimented with different values)
As there is less amount of data going for deep learning model is not correct


We can do the cross validation and grid search for tuning the hyperparameter like alpha of Lasso for getting good accuracy. But that kind of overfit the model.

So, we need to fit different regression models for different segment (Low spending Customers and High spending customers) otherwise logarithmic model will also do a good job but things get complicated when explaining the logaritmic values.

we can say that there is an increase in longterm value (LTV) if the observation period is increased for low end customers but for high end customers it is staying sttill

As expected the absolute error for a low spending customer is coming around 75(Long term Value < 700) and for high spending customers is coming around 857(1000< Long term Value < 16000).





In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import seaborn as sns
import time

In [12]:
dfp = pd.read_csv("Purchases.csv")

In [13]:
dfp.head()

,udid,date,daily_iap_rev,purchases_made
0,44080da2aaa54b95ae96327e3649b651,6/8/2015,3.5,1
1,6593f2ec3f6848afa1177198a7cf29b6,6/15/2015,14.0,1
2,7468b1857ea848b1aa6fece800193e52,7/13/2015,70.0,1
3,bf57de635791447eb630a998e8251a4a,8/25/2015,70.0,1
4,7468b1857ea848b1aa6fece800193e52,9/18/2015,70.0,1


In [14]:
dfp.isnull().sum()

udid              0
date              0
daily_iap_rev     0
purchases_made    0
dtype: int64

So, we do not have any missing values in our Purchases file

In [38]:
dfp["date"] = pd.to_datetime(dfp["date"])

In [39]:
np.sort(dfp[dfp["purchases_made"]==1]["daily_iap_rev"].unique())

array([ 3.5,  7. , 14. , 28. , 35. , 70. ])

So we can deduce that each inapp purchase ranges from 3.5 to 70.

In [40]:
dfp.gb = dfp.groupby("udid")

In [41]:
dfp.longtermvalue = dfp.gb["daily_iap_rev"].sum()

In [42]:
dfp.firstdate_of_purchase = dfp.gb["date"].min()

In [43]:
dfp.lastdate_of_purchase = dfp.gb["date"].max()

In [44]:
dfp.purchase_duration = pd.to_datetime(dfp.lastdate_of_purchase) - pd.to_datetime(dfp.firstdate_of_purchase) 
dfp.purchase_duration.head()

udid
0020f5c05795400bb2868009cf4b2080    62 days
002b934415cb4ae78579f3be87a6283d   173 days
004f2f923b8f47a6b9f91fa5c61095de     0 days
0076ecfa49cf4130870d52ed786dd8bc     0 days
00776d802fef4a26b525aed437af4728     0 days
Name: date, dtype: timedelta64[ns]

In [46]:
dfp.total_purchases_made = dfp.gb["purchases_made"].sum()

In [48]:
df_modified_Purchases = pd.DataFrame(dfp.longtermvalue.index,                                               
                                     columns = ["udid"],
                                     index = dfp.longtermvalue.index)

In [49]:

df_modified_Purchases["firstdate_of_purchase"] = dfp.firstdate_of_purchase
df_modified_Purchases["lastdate_of_purchase"] = dfp.lastdate_of_purchase
df_modified_Purchases["purchase_duration"] = dfp.purchase_duration
df_modified_Purchases["total_purchases_made"] = dfp.total_purchases_made
df_modified_Purchases["Long term Value"] = dfp.longtermvalue

In [50]:
df_modified_Purchases["purchase_duration"] = df_modified_Purchases["purchase_duration"]

In [51]:
df_modified_Purchases.head()

,udid,firstdate_of_purchase,lastdate_of_purchase,purchase_duration,total_purchases_made,Long term Value
udid,,,,,,
0020f5c05795400bb2868009cf4b2080,0020f5c05795400bb2868009cf4b2080,2015-06-01,2015-08-02,62 days,12,682.5
002b934415cb4ae78579f3be87a6283d,002b934415cb4ae78579f3be87a6283d,2015-06-06,2015-11-26,173 days,7,154.0
004f2f923b8f47a6b9f91fa5c61095de,004f2f923b8f47a6b9f91fa5c61095de,2015-06-10,2015-06-10,0 days,1,3.5
0076ecfa49cf4130870d52ed786dd8bc,0076ecfa49cf4130870d52ed786dd8bc,2015-09-11,2015-09-11,0 days,1,3.5
00776d802fef4a26b525aed437af4728,00776d802fef4a26b525aed437af4728,2015-06-06,2015-06-06,0 days,1,3.5


In [54]:
df_modified_Purchases["purchase_duration"] = str(df_modified_Purchases["purchase_duration"])


In [55]:
str_spl = lambda x:int(x.split()[0]) +1 
df_modified_Purchases["purchase_duration"]  = df_modified_Purchases["purchase_duration"].apply(str_spl) 

ValueError: invalid literal for int() with base 10: 'udid'